# SDM + PW
Single-diode model with the Photowatt-PWP201 dataset.

In [11]:
using PSO
using CUDA, Random

In [12]:
const data_dir = "../data/"
include("utils.jl")

benchmark (generic function with 1 method)

In [13]:
# set random seed: for reproduction
Random.seed!(CURAND.default_rng(), 123)

## Read data and set up search range

In [14]:
# select data type to be used on GPU: Float64 or Float32
# Float32 tends to be a little faster, but with lower precision
const TF = Float64

Float64

In [15]:
const T = TF(45 + 273.15)  # temperature in Kalvin
const data = read_data(joinpath(data_dir, "Photowatt25.csv"), TF);

In [16]:
const x_min = TF[0, 0,  1,  0, 0]
const x_max = TF[2, 50, 50, 2, 2000];

## PSO parameters

In [17]:
const w = TF(0.5)
const c1 = TF(2.5)
const c2 = TF(1.6)
const np = 3200
const niter = 200
const min_delta=TF(1e-10)
const patience=20
const threads=320  # threads per block in CUDA
;

## Run PSO
- Known best minimum in the literature: 1.47025e-04
- Note that the SSE (sum of square error) is minimized, which is equivalent to the minimization of RMSE.

In [18]:
@time pso(data, T, :SDM, x_min, x_max; niter, np, w, c1, c2, log=true, threads, min_delta, patience)

(threads, blocks) = (320, 10)
iter      sse       
1         3.53067e-01
2         6.67135e-02
3         5.59075e-02
4         1.92983e-02
5         6.26158e-04
6         6.26158e-04
7         6.26158e-04
8         4.79236e-04
9         4.79236e-04
10        4.19409e-04
11        2.61674e-04
12        2.10869e-04
13        2.10869e-04
14        1.98435e-04
15        1.86805e-04
16        1.86805e-04
17        1.86805e-04
18        1.80344e-04
19        1.72771e-04
20        1.72771e-04
21        1.72771e-04
22        1.72738e-04
23        1.72738e-04
24        1.72738e-04
25        1.72738e-04
26        1.72715e-04
27        1.72715e-04
28        1.72715e-04
29        1.72713e-04
30        1.72713e-04
31        1.72713e-04
32        1.72713e-04
33        1.72694e-04
34        1.72602e-04
35        1.72511e-04
36        1.72419e-04
37        1.72285e-04
38        1.72103e-04
39        1.71876e-04
40        1.71446e-04
41        1.70873e-04
42        1.70083e-04
43        1.69247e-04
44 

(sse = 0.00014702470290184583, solution = [1.0305142869508024, 3.4822779706921114, 48.64285148826236, 1.2012705793502625, 981.9853594675723], stopped_iter = 99)

## Measure statistics
Run PSO multiple times and record the results. Analyze the results.

In [19]:
f = () -> pso(data, T, :SDM, x_min, x_max; niter, np, w, c1, c2, log=false, threads, min_delta, patience)
measure_RMSE_stats(f, size(data, 2); n_trials=30)

Dict{Symbol, Float64} with 4 entries:
  :max  => 0.00242507
  :std  => 2.86476e-13
  :mean => 0.00242507
  :min  => 0.00242507

## Benchmark the running time
Run the PSO multiple times and measure the average running time. [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) is a library devoted to benchmarking in the `benchmark` function defined in `utils.jl`.

Due to GPU characteristics as well as early stopping, the variance among run time can be fairly high.

In [20]:
benchmark(f; n_trials=100)

BenchmarkTools.Trial: 
  memory estimate:  842.70 KiB
  allocs estimate:  18317
  --------------
  minimum time:     13.845 ms (0.00% GC)
  median time:      365.801 ms (0.00% GC)
  mean time:        252.779 ms (0.75% GC)
  maximum time:     496.661 ms (1.95% GC)
  --------------
  samples:          100
  evals/sample:     1